# Seldon Deployment CRD
> Learn about kubernetes custom resources, operators, and controllers through the seldon-core custom resource

- toc: true 
- badges: true
- comments: true
- categories: [kubernetes, docker]
- image: images/chart-preview.png

### Intro
In this post we will walk through the basics behind the seldon custom resource definition. 
To get started, let's get a cluster up and running. If you followed [part 1]() of this series, you can do so with kind. 

In [ ]:
!kind create cluster 

When working in kubernetes, I like to create a new namespace for that work. For those familiar with python, namespaces are similar to using virtual environments for different projects. Create a namespace with this command: 

In [ ]:
!kubectl create namespace seldon-intro

Then use kubens to set this as your current namespace:

In [ ]:
!kubens seldon-intro

### Install Seldon
I suggest using `helm` to install seldon. If you haven't used helm before, use [this page](https://helm.sh/) to find install instructions. (If you're on a mac, just use `brew install helm`). For those familiar with python, helm is like the pip for kubernetes. It works by using [helm charts](https://helm.sh/docs/topics/charts/). A chart is a group of files that describe a higher level application using built-in kubernetes resources. For example, you could use a helm chart to deploy a full stack web application. You can learn more about them [here](https://helm.sh/docs/topics/charts/).

Once helm is installed, use it to install seldon-core and seldon-core-operator with the following command:

In [ ]:
!helm install seldon-core seldon-core-operator \
    --repo https://storage.googleapis.com/seldon-charts \
    --set usageMetrics.enabled=true \
    --set ambassador.enabled=true \
    --namespace seldon-intro

In [ ]:
!kubectl get deployments

The docker image name used to create the seldon-controller-manager pod and deployment is called `docker.io/seldonio/seldon-core-operator:1.2.1`. That is because the seldon-controller-manager is the seldon-core-operator. 

Be sure to match the last line with the namespace you created in the previous command. We will talk more about what the other lines mean later in the series. 
After installing, run 

In [13]:
!kubectl get pods -o wide

NAME                                        READY   STATUS    RESTARTS   AGE   IP           NODE                 NOMINATED NODE   READINESS GATES
seldon-controller-manager-769694875-gk4gl   1/1     Running   0          18m   10.244.0.5   kind-control-plane   <none>           <none>


and you'll see you have a pod running with seldon-controller-manager in the name (followed by a random string, we will talk about why that is soon). Next, run 

In [12]:
!kubectl get deployments

NAME                        READY   UP-TO-DATE   AVAILABLE   AGE
seldon-controller-manager   1/1     1            1           18m


Seldon-controller-manager is a kubernetes operator. As described [here](https://www.jaegertracing.io/docs/1.16/operator/), kubernetes operators are extensions of the kubernetes api that allow users to easily package and deploy complex applications on kubernetes. The seldon-core-operator adds a lot of functionality to the kubernetes cluster and allows us to interact with seldon deployments as if they were a built-in kubernetes object. Along with this, we also installed the seldon-core custom resource definition. Kubernetes custom resources are extensions of the native kuberetes api. As you will see soon, we will now be able to deploy and interact with a new structure, the `sdep`, in the same fashion we deploy and monitor kubernetes deployments, servives, and other built-in resources. 


All interactions with a kubernetes cluster is the user specifying a new desired state, and the kubernetes cluster using it's resources to change into that desired state. When, a custom resource is created, an operator is needed as well to tell kubernetes how to handle updates to the desired state of the custom resource. 
The seldon-core operator is what allows users to make edits to currently running seldon deployments without downtime. 

Operators allows kubernetes to run stateful applications. A popular usecase for operators is databases. 

In [this post](https://enterprisersproject.com/article/2019/2/kubernetes-operators-plain-english?page=1), they write 
```
If you had to sum up Kubernetes in a word, the best choice might not be “orchestration” but “automation.” That’s what it’s all about: Kubernetes enables the automation of the infrastructure (and corresponding operational burden of managing that infrastructure) necessary for running containerized applications – a must when running these apps at scale in production environments.
```
This is very evident with the Seldon deployment CRD and operator. As we move along and start to create our own seldon deployments, you will see the power of this automation. With a single file, a seldon deployment will launch multiple services, deployments, pods, and allow for continuous updates of all those components through the Kubernetes api. Seldon has leveraged the power of the automation offered by kubernetes to create inferences graphs of many components. 

Another great quote from that article: 
```
“Operators are simplifying the process highly complex distributed database management by defining the installation, scale, updates, and management lifecycle of a stateful clustered application,” says Yossi Jana, DevOps team leader at AllCloud.

From another vantage point, consider life without Operators.

“Without Operators, many applications need intervention to deploy, scale, reconfigure, upgrade, or recover from faults,” Thompson says. “If your app – or apps that you depend on, such as your database management system – [requires] DevOps engineers hovering over a keyboard in these critical moments, hoping they get the steps correctly, you’re almost certain to have greater downtime and more stress in your team.”
```

As described in the kubernetes docs [here](https://kubernetes.io/docs/concepts/extend-kubernetes/operator/), operators follow the controller pattern, which means they are responsible for keep the desired state of the custom resource they are responsible for. 
The seldon-core-operator is responsbile for the Seldon Deployment custom resource. That means, when we create or edit a seldon deployment, the seldon-core-operator is responsible for adjusting the kubernetes deployment to the desired state to meet the new edits applied by the user. 


In every article you read about kubernetes operators, you'll find some sentiment to the fact that is confusing the first time around. In fact, pretty much everything you learn about kubernetes will be confusing the first time around. Don't let that discourage you. That is why we are using seldon to help our understanding. Instead of reading description after description of what a custom resource and operator are, let's actually use them to begin to understand the power they provide. 


In [27]:
%%bash
kubectl apply -f - << END
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: iris-model
spec:
  name: sklearn-iris-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/sklearn-iris:0.1
          imagePullPolicy: IfNotPresent
          name: sklearn-iris-classifier
    graph:
      children: []
      endpoint:
        type: REST
      name: sklearn-iris-classifier
      type: MODEL
    name: predictor
    replicas: 1
END

seldondeployment.machinelearning.seldon.io/iris-model configured


In [28]:
!kubectl get pods

NAME                                                              READY   STATUS    RESTARTS   AGE
iris-model-predictor-0-sklearn-iris-classifier-5f5b59678-8bpwk    0/2     Running   0          3s
iris-model-sklearn-iris-predictor-0-sklearn-iris-classifie4l84z   0/2     Running   0          17s
seldon-controller-manager-769694875-gk4gl                         1/1     Running   0          30m
